In [ ]:
%load_ext autoreload
%autoreload 2

## Generating CIPHER input files

This notebook has some usage examples for generating a CIPHER input file.

In [ ]:
from cipher_input import CIPHERInput, InterfaceDefinition
import plotly.express as px # we could use matplotlib here instead
import numpy as np

### 1. Random Voronoi tessellation of phases

These example generate the geometry using a Voronoi tessellation of a set of random seed point.

#### Example 1.1: One interface per material-pair

In [ ]:
# Define the material properties:
materials = {
    "mat1": {"chemicalenergy": "none"},
    "mat2": {"chemicalenergy": "none"},
}

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties={"energy": {"e0": 3}}
    ),
]

input_1_1 = CIPHERInput.from_random_voronoi(
    materials=materials,
    volume_fractions=[0.2, 0.8], # material volume fractions
    num_phases=500,
    grid_size=[128, 128, 128],
    size=[128, 128, 128],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={},
    interfaces=interfaces,
)

##### Write the input YAML file

In [ ]:
input_1_1.write_yaml("ex_1.1.yaml")

##### Visualise a slice of the phase map

In [ ]:
px.imshow(input_1_1.geometry.voxel_phase[20])

##### Visualise a slice of the phase interfaces (hiding bulk voxels)

In [ ]:
px.imshow(input_1_1.geometry.neighbour_voxels[20])

##### Visualise the interface map

This is the 2D symmetric matrix that CIPHER uses to assign each possible phase-pair to a given interface

In [ ]:
px.imshow(input_1_1.geometry.interface_map)

##### Visualise a slice of the interface indices of the interface voxels

In [ ]:
px.imshow(input_1_1.geometry.get_interface_idx()[20])

##### Visualise a slice of the material assignment

In [ ]:
px.imshow(input_1_1.geometry.voxel_material[20])

#### Example 1.2: Multiple interfaces types for a given phase-pair - equal distribution

In [ ]:
# Define the material properties:
materials = {
    "mat1": {"chemicalenergy": "none"},
    "mat2": {"chemicalenergy": "none"},
}

# Define the interfaces:
# "low-angle" and "high-angle" will be equally distributed for the mat1-mat1 interfaces
interfaces=[
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='low-angle',
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='high-angle',
        properties={"energy": {"e0": 2}},
    ),    
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties={"energy": {"e0": 3}}
    ),
]

input_1_2 = CIPHERInput.from_random_voronoi(
    materials=materials,
    volume_fractions=[0.2, 0.8], # material volume fractions
    num_phases=100,
    grid_size=[32, 32],
    size=[32, 32],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={},
    interfaces=interfaces,
)

In [ ]:
input_1_2.write_yaml("ex_1.2.yaml")

#### Example 1.3: Multiple interfaces types for a given phase-pair - specified distribution

In [ ]:
# Define the material properties:
materials = {
    "mat1": {"chemicalenergy": "none"},
    "mat2": {"chemicalenergy": "none"},
}

# Define the interfaces:
# "low-angle" and "high-angle" will be distributed according to `type_fraction`
interfaces=[
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='low-angle',
        type_fraction=0.7,
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        type_label='high-angle',
        type_fraction=0.3,
        properties={"energy": {"e0": 2}},
    ),    
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties={"energy": {"e0": 3}}
    ),
]

input_1_3 = CIPHERInput.from_random_voronoi(
    materials=materials,
    volume_fractions=[0.9, 0.1], # material volume fractions
    num_phases=100,
    grid_size=[32, 32],
    size=[32, 32],
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={},
    interfaces=interfaces,
)

input_1_3.write_yaml("ex_1.3.yaml")

### 2. Voronoi tessellation of existing seed points

#### Example 2.1: using pre-existing seed positions for the Voronoi tessellation

For this, we just use `CIPHERInput.from_seed_voronoi` instead of `CIPHERInput.from_random_voronoi`, and pass `seeds` instead of `num_phases`, where `seeds` should be an `(N, 2)` or `(N, 3)` array for 2D or 3D, respectively. Seeds are specified in real-space units, so must be defined within `size`.

In [ ]:
# Here we define some seeds using CIPHERGeometry, but may define seeds in some other way.

from cipher_gen import CIPHERGeometry

size = [128, 128]
seeds = CIPHERGeometry.get_random_seeds(num_phases=50, size=size)

# visualise the seeds:
px.scatter(x=seeds[:, 0], y=seeds[:, 1])

In [ ]:
# Define the material properties:
materials = {
    "mat1": {"chemicalenergy": "none"},
    "mat2": {"chemicalenergy": "none"},
}

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties={"energy": {"e0": 3}}
    ),
]

input_2_1 = CIPHERInput.from_seed_voronoi(
    materials=materials,
    volume_fractions=[0.2, 0.8],
    seeds=seeds,
    grid_size=[128, 128],
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={},
    interfaces=interfaces,
)

input_2_1.write_yaml("ex_2.1.yaml")

### 3. Passing in an existing voxel-phase map

We can pass in directly the voxel map if we have it, using `CIPHERInput.from_voxel_phase_map`.

In [ ]:
# First let's generate a simple 2D voxel phase map. This could be generated in some other way.
from discrete_voronoi import DiscreteVoronoi

size = [64, 64]
num_phases = 10
voronoi_obj = DiscreteVoronoi.from_random(
    size=size,
    grid_size=[64, 64],
    num_regions=num_phases,
)
voxel_phase = voronoi_obj.region_ID

# visualise the voxel_phase map:
px.imshow(voxel_phase)

#### Example 3.1: phase-material assignment using specified volume fractions

In [ ]:
# Define the material properties:
materials = {
    "mat1": {"chemicalenergy": "none"},
    "mat2": {"chemicalenergy": "none"},
}

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties={"energy": {"e0": 3}}
    ),
]

input_3_1 = CIPHERInput.from_voxel_phase_map(
    voxel_phase=voxel_phase,
    materials=materials,
    volume_fractions=[0.2, 0.8],
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={},
    interfaces=interfaces,
)

input_3_1.write_yaml("ex_3.1.yaml")

In [ ]:
px.imshow(input_3_1.geometry.voxel_phase)

In [ ]:
px.imshow(input_3_1.geometry.voxel_material)

#### Example 3.2: specify phase_material assignment as well

Instead of using `volume_fractions`, here we specify directly which materials each phase belongs to, using the `phase_material` parameter.

In [ ]:
# Define the material properties:
materials = {
    "mat1": {"chemicalenergy": "none"},
    "mat2": {"chemicalenergy": "none"},
}

# Generate a phase material mapping. This could be done in some other way.
rng = np.random.default_rng()
phase_material = rng.choice(a=len(materials), size=num_phases)
print(phase_material)

In [ ]:
# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        materials=("mat1", "mat2"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat1", "mat1"),
        properties={"energy": {"e0": 1}},
    ),
    InterfaceDefinition(
        materials=("mat2", "mat2"),
        properties={"energy": {"e0": 3}}
    ),
]

input_3_2 = CIPHERInput.from_voxel_phase_map(
    voxel_phase=voxel_phase,
    materials=materials,
    phase_material=phase_material,
    size=size,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={},
    interfaces=interfaces,
)

input_3_2.write_yaml("ex_3.2.yaml")

In [ ]:
px.imshow(input_3_2.geometry.voxel_phase)